In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
%cd /content/drive/MyDrive/CVL/cyclediffusion_gitcode/

[Errno 2] No such file or directory: '/content/drive/MyDrive/CVL/cyclediffusion_gitcode/'
/home/anna/Documents/cyclediffusion_gitcode/notebooks


In [2]:
import os
os.getcwd()
os.chdir('/home/anna/Documents/cyclediffusion_gitcode')

In [4]:
# set the device
import torch
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
  print('Good to go!')

Good to go!


In [20]:
device

device(type='cuda', index=1)

## Preparation

In [3]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import sys
from models.pretrained.pix2struct import processor, model
from src.training.training_utils import collator, cycle_collator
from transformers import AdamW

/home/anna/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import pickle

# save the dataset to a file
def save_dataset(dataset, filename):
    with open(filename, 'wb') as file:
        pickle.dump(dataset, file)

# load the dataset from a file
def load_dataset(filename):
    with open(filename, 'rb') as file:
        return pickle.load(file)

In [5]:
subset_A_reverseSVO = load_dataset('subset_A_reverseSVO.pkl')

In [6]:
import numpy as np

labels_A = set()

for idx in range(len(subset_A_reverseSVO)):
    item = subset_A_reverseSVO[idx]
    label = item["label"]
    labels_A.add(label)

labels_A = list(labels_A)

In [9]:
print(labels_A[:8])

['a square hanging over a hexagon', 'a rectangle hanging over a ellipse', 'a parallelogram on a ellipse', 'a square on the right of a circle', 'a ellipse on the right of a pentagon', 'a triangle under a rectangle', 'a rectangle hanging over a parallelogram', 'a ellipse on the left of a pentagon']


In [9]:
len(labels_A)

168

In [10]:
subset_B_reverseSVO = load_dataset('subset_B_reverseSVO.pkl')

# Exp 1: subset A only pixel loss

In [7]:
from models.cyclediffusion_dp import CycleDiffusionModel
from src.data_preparation.twoD_image_dataset import combine_datasets_reverse, combine_datasets_newSO, split_dataset, SubsetDataset

In [8]:
import torch.optim as optim
import torch.nn as nn
import torch
import tqdm
# from src.data_preparation.captions_dataset import ds_train, ds_test
from torch.utils.data import DataLoader
from src.training.training_utils import cycle_collator
from torch.utils.tensorboard import SummaryWriter
from torch.cuda.amp import GradScaler, autocast
import os

In [17]:
from src.training.train_cyclediffusion_dp import train_cyclediff

In [10]:
train_indices, val_indices = split_dataset(subset_A_reverseSVO, val_size=0.1)
ds_train = SubsetDataset(subset_A_reverseSVO, train_indices)
ds_test = SubsetDataset(subset_A_reverseSVO, val_indices)

train_dataloader = DataLoader(ds_train, batch_size=1, shuffle=True, num_workers=4, pin_memory=True, collate_fn=cycle_collator)
val_dataloader = DataLoader(ds_test, batch_size=1, shuffle=True, num_workers=4, pin_memory=True, collate_fn=cycle_collator)

In [11]:
# model_1 = CycleDiffusionModel(verbose=False).split_models()
model_1 = CycleDiffusionModel(verbose=False)
optimizer = optim.Adam(model_1.parameters(), lr=0.001)

The config attributes {'predict_epsilon': True} were passed to DDPMScheduler, but are not expected and will be ignored. Please verify your scheduler_config.json configuration file.
An error occurred while trying to fetch OFA-Sys/small-stable-diffusion-v0: OFA-Sys/small-stable-diffusion-v0 does not appear to have a file named diffusion_pytorch_model.safetensors.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch OFA-Sys/small-stable-diffusion-v0: OFA-Sys/small-stable-diffusion-v0 does not appear to have a file named diffusion_pytorch_model.safetensors.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


In [16]:
# import torch.quantization

# # Custom function to apply qconfig selectively
# def apply_qconfig_recursive(module):
#     for name, child in module.named_children():
#         # Skip quantization for modules that should not be quantized
#         if isinstance(child, (nn.Embedding, nn.LayerNorm, nn.Softmax)):
#             child.qconfig = None  # Skip quantization for these layers
#         elif isinstance(child, (nn.Linear, nn.Conv2d)):
#             # Apply different qconfigs based on the layer type
#             if isinstance(child, nn.Embedding):
#                 child.qconfig = torch.quantization.float_qparams_weight_only_qconfig
#             else:
#                 child.qconfig = torch.quantization.get_default_qconfig('fbgemm')
#         apply_qconfig_recursive(child)

# # Apply quantization to the entire model
# def quantize_model(model):
#     model.eval()  
#     apply_qconfig_recursive(model)
#     model_prepared = torch.quantization.prepare(model, inplace=False)
#     model_quantized = torch.quantization.convert(model_prepared, inplace=False)

#     return model_quantized


In [17]:
# model_1 = quantize_model(model_1)

In [18]:
# train_dataloader = DataLoader(ds_train, batch_size=1, shuffle=True, collate_fn=cycle_collator)
# val_dataloader = DataLoader(ds_test, batch_size=1, shuffle=True, collate_fn=cycle_collator)

with torch.autograd.profiler.profile() as prof:
    train_cyclediff(model=model_1, condition=1, labels_A=labels_A, optimizer=optimizer, train_dataloader=train_dataloader, 
                val_dataloader=val_dataloader, epochs=1000, patience=3, accumulation_steps=4)
print(prof.key_averages().table(sort_by="cuda_memory_usage"))


[2024-08-22 18:58:58,579] [INFO] [logging.py:96:log_dist] [Rank -1] DeepSpeed info: version=0.14.5, git-hash=unknown, git-branch=unknown
[2024-08-22 18:58:58,579] [INFO] [comm.py:637:init_distributed] cdb=None
[2024-08-22 18:58:58,580] [INFO] [comm.py:652:init_distributed] Not using the DeepSpeed or dist launchers, attempting to detect MPI environment...


STAGE:2024-08-22 18:58:58 3810726:3810726 ActivityProfilerController.cpp:314] Completed Stage: Warm Up


[2024-08-22 18:58:59,148] [INFO] [comm.py:702:mpi_discovery] Discovered MPI settings of world_rank=0, local_rank=0, world_size=1, master_addr=164.67.77.58, master_port=29500
[2024-08-22 18:58:59,149] [INFO] [comm.py:668:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl
[2024-08-22 18:59:01,110] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed Flops Profiler Enabled: False
[2024-08-22 18:59:01,112] [INFO] [logging.py:96:log_dist] [Rank 0] Using client Optimizer as basic optimizer
[2024-08-22 18:59:01,112] [INFO] [logging.py:96:log_dist] [Rank 0] Removing param_group that has no 'params' in the basic Optimizer
[2024-08-22 18:59:01,172] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed Basic Optimizer = Adam
[2024-08-22 18:59:01,173] [INFO] [utils.py:59:is_zero_supported_optimizer] Checking ZeRO support for optimizer=Adam type=<class 'torch.optim.adam.Adam'>
[2024-08-22 18:59:01,173] [INFO] [logging.py:96:log_dist] [Rank 0] Creating torch.float16 ZeRO stage 2 o

  0%|                                                   | 0/604 [00:00<?, ?it/s]/home/anna/.local/lib/python3.8/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,

  0%|                                                   | 0/604 [00:04<?, ?it/s]
STAGE:2024-08-22 18:59:07 3810726:3810726 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2024-08-22 18:59:07 3810726:3810726 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 

In [12]:
torch.cuda.empty_cache()

In [19]:
!nvidia-smi

Thu Aug 22 18:59:35 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3080 Ti      Off| 00000000:01:00.0 Off |                  N/A |
|  0%   78C    P8               52W / 400W|  11963MiB / 12288MiB |     22%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [21]:
import os
os.getpid()

1231155

In [ ]:
!kill -9 1520470

In [35]:
torch.__version__

'2.3.0+cu121'

In [37]:
import deepspeed
deepspeed.__version__

'0.14.5'